# Data analysis

In this notebook, I will present the data analysis I performed on the database to gain a deeper understanding of its structure and contents, enabling me to leverage it effectively in building my recommender systems.

## Setup

In [1]:
%%bash
DATA_FOLDER="../data"
if [ ! -d "$DATA_FOLDER" ]; then
    wget --no-check-certificate "https://drive.usercontent.google.com/download?id=1qe5hOSBxzIuxBb1G_Ih5X-O65QElollE&export=download&confirm=t&uuid=b2002093-cc6e-4bd5-be47-9603f0b33470" -O KuaiRec.zip
    unzip KuaiRec.zip -d "$DATA_FOLDER"
    rm KuaiRec.zip
fi

In [2]:
import pandas as pd
import json

In [3]:
data = "../data/KuaiRec 2.0/data"
small_matrix = pd.read_csv(f"{data}/small_matrix.csv")
big_matrix = pd.read_csv(f"{data}/big_matrix.csv")
item_categories = pd.read_csv(f"{data}/item_categories.csv")
item_features = pd.read_csv(f"{data}/item_daily_features.csv")
social_network = pd.read_csv(f"{data}/social_network.csv")
user_features = pd.read_csv(f"{data}/user_features.csv")

dfs = {
    "Small Matrix": small_matrix,
    "Big Matrix": big_matrix,
    "Item Categories": item_categories,
    "Item Features": item_features,
    "Social Network": social_network,
    "User Features": user_features
}

In [4]:
# Convert the feat column from a string representation of an array into an array.
item_categories["feat"] = item_categories["feat"].apply(lambda x: json.loads(x))

## Table infos

### Shapes

In [5]:
for name, df in dfs.items():
    print(f"{name} shape: {df.shape}")

Small Matrix shape: (4676570, 8)
Big Matrix shape: (12530806, 8)
Item Categories shape: (10728, 2)
Item Features shape: (343341, 58)
Social Network shape: (472, 2)
User Features shape: (7176, 31)


### Missing values

In [6]:
for name, df in dfs.items():
    print(f"Missing Values in {name}:")
    print(df.isnull().sum())
    print("---------------------------")

Missing Values in Small Matrix:
user_id                0
video_id               0
play_duration          0
video_duration         0
time              181992
date              181992
timestamp         181992
watch_ratio            0
dtype: int64
---------------------------
Missing Values in Big Matrix:
user_id           0
video_id          0
play_duration     0
video_duration    0
time              0
date              0
timestamp         0
watch_ratio       0
dtype: int64
---------------------------
Missing Values in Item Categories:
video_id    0
feat        0
dtype: int64
---------------------------
Missing Values in Item Features:
video_id                        0
date                            0
author_id                       0
video_type                      0
upload_dt                       0
upload_type                     0
visible_status                  0
video_duration              10598
video_width                     0
video_height                    0
music_id           

### Duplicate values

In [7]:
for name, df in dfs.items():
    # Skip Item Categories since it contains lists.
    if name != "Item Categories":
        print(f"Duplicate Values in {name}:")
        print(df.duplicated().sum())
        print("---------------------------")

Duplicate Values in Small Matrix:
0
---------------------------
Duplicate Values in Big Matrix:
965819
---------------------------
Duplicate Values in Item Features:
0
---------------------------
Duplicate Values in Social Network:
0
---------------------------
Duplicate Values in User Features:
0
---------------------------


### Small matrix

In [8]:
print(f"Unique Users: {small_matrix['user_id'].nunique()}")
print(f"Unique Items: {small_matrix['video_id'].nunique()}")

Unique Users: 1411
Unique Items: 3327


In [9]:
small_matrix.groupby("user_id").agg(
    video_count=("video_id", "count"),
    avg_watch_ratio=("watch_ratio", "mean"),
    total_play_time=("play_duration", "sum"),
    complete_views=("watch_ratio", lambda x: sum(x >= 1)),
).reset_index()

,user_id,video_count,avg_watch_ratio,total_play_time,complete_views
0,14,3321,0.997809,31832319,1301
1,19,3313,0.836359,26213861,934
2,21,3307,0.947653,29482494,1222
3,23,3304,1.024486,32613376,1079
4,24,3315,0.864112,27455866,883
...,...,...,...,...,...
1406,7142,3319,0.799392,25264885,850
1407,7147,3322,0.931653,29176212,1301
1408,7153,3316,0.814056,25230922,753
1409,7159,3319,0.779488,24701044,984


### Big matrix

In [10]:
print(f"Unique Users: {big_matrix['user_id'].nunique()}")
print(f"Unique Items: {big_matrix['video_id'].nunique()}")

Unique Users: 7176
Unique Items: 10728


In [11]:
big_matrix.groupby("user_id").agg(
    video_count=("video_id", "count"),
    avg_watch_ratio=("watch_ratio", "mean"),
    total_play_time=("play_duration", "sum"),
    complete_views=("watch_ratio", lambda x: sum(x >= 1)),
).reset_index()

,user_id,video_count,avg_watch_ratio,total_play_time,complete_views
0,0,2513,1.135499,25788848,1286
1,1,1504,0.949711,14405565,508
2,2,1064,0.648013,7177532,174
3,3,3456,0.884631,27916071,1099
4,4,584,0.821825,6171273,206
...,...,...,...,...,...
7171,7171,1398,1.006527,16442646,380
7172,7172,2275,1.094206,25662760,741
7173,7173,830,1.094328,10292019,196
7174,7174,1823,0.756397,13494239,409


### User features

In [12]:
print(f"Total Users: {len(user_features)}")
user_features

Total Users: 7176


,user_id,user_active_degree,is_lowactive_period,is_live_streamer,is_video_author,follow_user_num,follow_user_num_range,fans_user_num,fans_user_num_range,friend_user_num,...,onehot_feat8,onehot_feat9,onehot_feat10,onehot_feat11,onehot_feat12,onehot_feat13,onehot_feat14,onehot_feat15,onehot_feat16,onehot_feat17
0,0,high_active,0,0,0,5,"(0,10]",0,0,0,...,184,6,3,0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,full_active,0,0,0,386,"(250,500]",4,"[1,10)",2,...,186,6,2,0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,full_active,0,0,0,27,"(10,50]",0,0,0,...,51,2,3,0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,full_active,0,0,0,16,"(10,50]",0,0,0,...,251,3,2,0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,full_active,0,0,0,122,"(100,150]",4,"[1,10)",0,...,99,4,2,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7171,7171,full_active,0,0,1,52,"(50,100]",1,"[1,10)",0,...,259,1,4,0,1.0,0.0,0.0,0.0,0.0,0.0
7172,7172,full_active,0,0,0,45,"(10,50]",2,"[1,10)",2,...,11,2,0,0,1.0,0.0,0.0,0.0,0.0,0.0
7173,7173,full_active,0,0,0,615,500+,3,"[1,10)",2,...,51,2,2,0,1.0,0.0,0.0,0.0,0.0,0.0
7174,7174,full_active,0,0,0,959,500+,0,0,0,...,107,3,2,0,0.0,0.0,0.0,0.0,0.0,0.0


### Item features

In [13]:
print(f"Total Items: {item_features['video_id'].nunique()}")

agg_columns = []
other_columns = [col for col in item_features.columns if col != "video_id" and col not in agg_columns]

agg_dict = {}
for col in other_columns:
    if "cnt" in col or "num" in col:
        agg_dict[col] = "sum"
    else:
        agg_dict[col] = "last"

item_features.groupby("video_id").agg(
    agg_dict
)

Total Items: 10728


,date,author_id,video_type,upload_dt,upload_type,visible_status,video_duration,video_width,video_height,music_id,...,download_cnt,download_user_num,report_cnt,report_user_num,reduce_similar_cnt,reduce_similar_user_num,collect_cnt,collect_user_num,cancel_collect_cnt,cancel_collect_user_num
video_id,,,,,,,,,,,,,,,,,,,,,
0,20200905,3309,NORMAL,2020-03-30,ShortImport,public,5966.0,720,1280,3350323409,...,216,215,2,1,151,151,119.0,116.0,90.0,89.0
1,20200905,4978,NORMAL,2020-04-09,PictureSet,public,NaN,886,1015,1812462382,...,77,51,0,0,125,121,0.0,0.0,7.0,7.0
2,20200905,939,NORMAL,2020-04-11,Kmovie,public,8000.0,720,1280,0,...,41,41,0,0,208,202,0.0,0.0,0.0,0.0
3,20200905,5889,NORMAL,2020-04-11,PictureSet,public,NaN,1080,1080,0,...,0,0,0,0,3,3,3.0,3.0,11.0,11.0
4,20200904,4284,NORMAL,2020-04-12,ShortCamera,public,18000.0,720,1280,3442844592,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10723,20200905,236,NORMAL,2020-09-05,ShortImport,public,4833.0,720,1280,4428603493,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
10724,20200905,5271,NORMAL,2020-09-05,LongImport,public,54720.0,720,1280,1090207430,...,1,1,0,0,0,0,0.0,0.0,0.0,0.0
10725,20200905,1924,NORMAL,2020-09-05,ShortImport,public,15800.0,576,1024,4429406509,...,5,5,0,0,4,4,0.0,0.0,0.0,0.0


### Item categories

In [14]:
print(f"Distinct features: {pd.Series([feat for feats in item_categories['feat'] for feat in feats]).nunique()}")
item_categories

Distinct features: 31


,video_id,feat
0,0,[8]
1,1,"[27, 9]"
2,2,[9]
3,3,[26]
4,4,[5]
...,...,...
10723,10723,[11]
10724,10724,[2]
10725,10725,[15]
10726,10726,[19]
